In [23]:
get_ipython().system('pip install htrc-feature-reader')
get_ipython().system('pip install pyLDAvis')
get_ipython().system('pip install htrc')

  Using cached pandas-1.5.3-cp38-cp38-macosx_10_9_x86_64.whl (11.9 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
htrc 0.1.58 requires pandas==1.1.5, but you have pandas 1.5.3 which is incompatible.
  Using cached pandas-1.1.5-cp38-cp38-macosx_10_9_x86_64.whl (10.1 MB)
  Using cached argparse-1.1-py3-none-any.whl


  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.3.1 requires pandas>=1.2.0, but you have pandas 1.1.5 which is incompatible.


In [2]:
import pandas as pd
from htrc_features import FeatureReader, Volume
from tqdm.notebook import trange, tqdm
import os
import shutil
import nltk
#import gensim

# Because some of our libraries will throw many, many warnings for future changes to their code:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
os.chdir('/Users/tomwilliams/Library/Mobile Documents/com~apple~CloudDocs/UVA/Thesis/Code')

In [4]:
from htrc import workset
bookerVolIds = workset.load_hathitrust_collection('https://babel.hathitrust.org/cgi/mb?a=listis&c=1335516225')
FeatureReader(ids=bookerVolIds).first().title#This checks first title in the list
#This creates a list of volume IDs for me to work with

'The 27th Kingdom /'

In [17]:
x = bookerVolIds[0:10]
print(x)

['uc1.b4948549', 'uc1.32106018081080', 'uc1.b4948376', 'mdp.39015040702071', 'mdp.39015059246804', 'mdp.39015058209969', 'mdp.49015002801653', 'uva.x001844478', 'mdp.39015043101354', 'mdp.39015068802894']


In [5]:
def ef_vol_to_bow_df(volume, save_to_tsv=False):
    from htrc_features import FeatureReader, Volume
    import pandas as pd
    from tqdm.notebook import trange, tqdm
    import os
    notebook_path = os.getcwd()
    htid = volume.id
    # creating an empty Dataframe to which we'll add clean tokens, instead of a text file
    vol_df = pd.DataFrame(columns=['htid', 'page_number', 'page_tokens'])
    outfile_name = htid+'.tsv' # saving our volume-level token DataFrame to a TSV
    for page in tqdm(volume.pages(), total=volume.page_count):
        page_num = str(page).split(' ')[1]
        page_df = page.tokenlist(section='body', case=False, pos=False)
        tkn_list = []
        
        for i, r in page_df.iterrows():
            tkn = i[2]
            clean_tkn = tkn.strip()
            count = r[0]
            tkns = ([f'{clean_tkn}'] * count)
            clean_tkn_list = [word for word in tkns if word.isalpha()]
            clean_tkn_list = [word for word in clean_tkn_list if word not in en_stop]
            tkn_list.extend(clean_tkn_list)
        '''
        Instead of writing to text files, we are adding the page-level clean tokens to our
        DataFrame, with one each page of tokens constituting one row in the DataFrame.
        '''
        vol_df = vol_df.append({'htid': htid, 'page_number':  page_num, 'page_tokens': tkn_list}, ignore_index=True)
    '''
    Lastly, we save our volume-level DataFrame as a tab-separated file, and return the volume 
    DataFrame so that we can better aggregate each volume's tokens into a single DataFrame (you'll 
    see this code in the wrapper we write to iterate through multiple volumes)
    '''
    if save_to_tsv==True:
        vol_df.to_csv(outfile_name, sep='\t', index=False)
        print(f'Saved {volume.title} to TSV named {outfile_name}')
    
    print(f'Reformatted "{volume.title}" ({htid}) to bag-of-words')
    return vol_df

In [6]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

en_stop.add("'")
en_stop.add('"')
en_stop.add(' ')
en_stop.add('would')
en_stop.add('could')
en_stop.add('should')
en_stop.add('said')
en_stop.add('also')

# If we wanted to be a bit more clever, we could use a simple loop to add these words to our stop list with less typing:

# stop_words_to_add = ["'",'"',' ','would','could','should','also', 'said']

# for word in stop_words_to_add:
#     en_stop.add(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tomwilliams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
workset_page_df = pd.DataFrame(columns=['htid','page_number','page_tokens'])

for book in tqdm(bookerVolIds):
    fr_vol = Volume(book)
    book_df = ef_vol_to_bow_df(fr_vol, save_to_tsv=True)
    workset_page_df = workset_page_df.append(book_df)
    
print(f"Reformatted {len(volume_list)} volumes to bag-of-words pages.")



Saved The 27th Kingdom / to TSV named uc1.b4948549.tsv
Reformatted "The 27th Kingdom /" (uc1.b4948549) to bag-of-words



Saved The accidental / to TSV named uc1.32106018081080.tsv
Reformatted "The accidental /" (uc1.32106018081080) to bag-of-words



Saved According to Mark / to TSV named uc1.b4948376.tsv
Reformatted "According to Mark /" (uc1.b4948376) to bag-of-words



Saved Alias Grace / to TSV named mdp.39015040702071.tsv
Reformatted "Alias Grace /" (mdp.39015040702071) to bag-of-words



Saved All for love / to TSV named mdp.39015059246804.tsv
Reformatted "All for love /" (mdp.39015059246804) to bag-of-words


...


Saved The black prince. to TSV named mdp.39015004264456.tsv
Reformatted "The black prince." (mdp.39015004264456) to bag-of-words



Saved The blue tango / to TSV named uc1.32106016735224.tsv
Reformatted "The blue tango /" (uc1.32106016735224) to bag-of-words



Saved Bomber; events relating to the last flight of an R.A.F. [bomber] over Germany on the night of June 31, 1943. to TSV named uc1.b4948541.tsv
Reformatted "Bomber; events relating to the last flight of an R.A.F. [bomber] over Germany on the night of June 31, 1943." (uc1.b4948541) to bag-of-words


ERROR:root:HTTP Error accessing http://data.analytics.hathitrust.org/features-2020.03/ucbk/a+82234/ucbk.ark+=28722=h2tx35d4d.json.bz2
Traceback (most recent call last):
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/site-packages/htrc_features/caching.py", line 73, in open
    fout = super().open(id, **kwargs)
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/site-packages/htrc_features/resolvers.py", line 121, in open
    uncompressed = self._open(id = id, suffix = suffix, mode = mode, format = format,
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/site-packages/htrc_features/resolvers.py", line 203, in _open
    return Path(dir, filename).open(mode = mode)
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/pathlib.py", line 1221, in open
    return io.open(self, mode, buffering, encoding, errors, newline,
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/pathlib.py", line 1077, in _opener
    return self._accessor.open(self, flags, mode)
FileNotFoundErro

HTTPError: HTTP Error 404: Not Found

In [11]:
x = bookerVolIds[32]
print(x)

ucbk.ark:/28722/h2tx35d4d


In [18]:
y = bookerVolIds[150]
print(y)


ucbk.ark:/28722/h24m91c3s


In [9]:
z = bookerVolIds[181]
print(z)

uc1.31158007693160


In [10]:
a = bookerVolIds[215]
print(a)

uc1.32106014751256


In [28]:
#bookerVolIds.remove(x)
#bookerVolIds.remove(y)
#bookerVolIds.remove(z)
#bookerVolIds.remove(a)

#These 4 volumes caused the next cell to crash. There are

print(x,y,z,a)

ucbk.ark:/28722/h2tx35d4d ucbk.ark:/28722/h24m91c3s ucbk.ark:/28722/h27940w7z ucbk.ark:/28722/h2x34n702


In [26]:
workset_page_df = pd.DataFrame(columns=['htid','page_number','page_tokens'])

for book in tqdm(bookerVolIds):
    fr_vol = Volume(book)
    book_df = ef_vol_to_bow_df(fr_vol, save_to_tsv=True)
    workset_page_df = workset_page_df.append(book_df)
    
print(f"Reformatted {len(volume_list)} volumes to bag-of-words pages.")


Saved The 27th Kingdom / to TSV named uc1.b4948549.tsv
Reformatted "The 27th Kingdom /" (uc1.b4948549) to bag-of-words



Saved The accidental / to TSV named uc1.32106018081080.tsv
Reformatted "The accidental /" (uc1.32106018081080) to bag-of-words



Saved According to Mark / to TSV named uc1.b4948376.tsv
Reformatted "According to Mark /" (uc1.b4948376) to bag-of-words



Saved Alias Grace / to TSV named mdp.39015040702071.tsv
Reformatted "Alias Grace /" (mdp.39015040702071) to bag-of-words



Saved All for love / to TSV named mdp.39015059246804.tsv
Reformatted "All for love /" (mdp.39015059246804) to bag-of-words


...


Saved The wilderness / to TSV named mdp.39015078779694.tsv
Reformatted "The wilderness /" (mdp.39015078779694) to bag-of-words



Saved Winnie and Wolf / to TSV named mdp.39015070739423.tsv
Reformatted "Winnie and Wolf /" (mdp.39015070739423) to bag-of-words



Saved Wolfy and the strudelbakers / to TSV named mdp.39015053778448.tsv
Reformatted "Wolfy and the strudelbakers /" (mdp.39015053778448) to bag-of-words



Saved Yellow dog / to TSV named mdp.39015057591474.tsv
Reformatted "Yellow dog /" (mdp.39015057591474) to bag-of-words



NameError: name 'volume_list' is not defined

In [7]:
missingVolIds = ["ucbk.ark+=28722=h24m91c3s", "ucbk.ark+=228722=h27940w7z","ucbk.ark+=228722=h2tx35d4d","ucbk.ark+=228722=h2x34n702"]

In [8]:
print(missingVolIds)

['ucbk.ark+=28722=h24m91c3s', 'ucbk.ark+=228722=h27940w7z', 'ucbk.ark+=228722=h2tx35d4d', 'ucbk.ark+=228722=h2x34n702']


In [9]:
workset_page_df2 = pd.DataFrame(columns=['htid','page_number','page_tokens'])

for book in tqdm(missingVolIds):
    fr_vol = Volume(book)
    book_df = ef_vol_to_bow_df(fr_vol, save_to_tsv=True)
    workset_page_df = workset_page_df2.append(book_df)
    
print(f"Reformatted {len(volume_list)} volumes to bag-of-words pages.")


ERROR:root:HTTP Error accessing http://data.analytics.hathitrust.org/features-2020.03/ucbk/a+82293/ucbk.ark+=28722=h24m91c3s.json.bz2
Traceback (most recent call last):
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/site-packages/htrc_features/caching.py", line 73, in open
    fout = super().open(id, **kwargs)
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/site-packages/htrc_features/resolvers.py", line 121, in open
    uncompressed = self._open(id = id, suffix = suffix, mode = mode, format = format,
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/site-packages/htrc_features/resolvers.py", line 203, in _open
    return Path(dir, filename).open(mode = mode)
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/pathlib.py", line 1221, in open
    return io.open(self, mode, buffering, encoding, errors, newline,
  File "/Users/tomwilliams/opt/anaconda3/lib/python3.8/pathlib.py", line 1077, in _opener
    return self._accessor.open(self, flags, mode)
FileNotFoundErro

HTTPError: HTTP Error 404: Not Found